This just ellucidates and confirms that our effnet is doing what we expected

In [1]:
from data import *
from model import *

import torch
import numpy as np
import pandas as pd
import os
import h5py
from exabiome.nn.loader import read_dataset, LazySeqDataset, train_test_loaders
import argparse
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from typing import Type, Any, Callable, Union, List, Optional

In [2]:
hparams = argparse.Namespace(**{'load': False,
                            'window': 4096,
                            'step': 4096,
                             'classify': False,
                               'tgt_tax_lvl': "species",
                               'fwd_only': True,
                               'manifold': True})

In [3]:
path = '/global/homes/a/azaidi/ar122_r202.toy.input.h5'
chunks = LazySeqDataset(hparams, path=path,
                       keep_open=True)

In [4]:
def get_toy_dl(hparams, batch_size=16):
    path = '/global/homes/a/azaidi/ar122_r202.toy.input.h5'
    chunks = LazySeqDataset(hparams, path=path,
                           keep_open=True)
    ds = taxon_ds(chunks, old_pad_seq)
    return DataLoader(ds, batch_size=batch_size, 
                      shuffle=True), ds

In [5]:
dl,ds = get_toy_dl(hparams, batch_size=8)
batch = next(iter(dl))
len(dl)

2377

In [9]:
train_dl, val_dl, test_dl = train_test_loaders(chunks, batch_size=16, distances=True)
batch = next(iter(train_dl))
len(batch)

5

Let's make sure to use the data/target that will match 

In [10]:
x = batch[1]
y = batch[2]
x.shape, y.shape

(torch.Size([16, 4096]), torch.Size([16, 16]))

In [12]:
p_list = [
        [[16,96,3,24,3,2,1], [24,144,6,24,3,1,1]], #layer 1
        [[24,144,6,40,5,2,2], [40,240,10,40,5,1,2]], #layer 2 
        [[40,240,10,80,3,2,1],[80,480,20,80,3,1,1], 
                                [80,480,20,80,3,1,1]], #layer 3
        [[80,480,20,112,5,1,2], [112,672,28,112,5,1,2], 
                                 [112,672,28,112,5,1,2]], #layer 4
        [[112,672,28,192,5,2,2],[192,1152,48,192,5,1,2],
         [192,1152,48,192,5,1,2], [192,1152,48,192,5,1,2]], #layer 5
        [[192,1152,48,320,3,2,1]] #layer 6
         ]

In [119]:
model = EffNet_b0(p_list, out_feats=1, n_classes=16)

In [120]:
out = model(x.unsqueeze(1).to(torch.float))
out.shape, y.shape

(torch.Size([16, 16, 1]), torch.Size([16, 16]))

Looks like the forward pass does what's expected - now to just confirm with the loss function

In [118]:
class DistMSELoss(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, output, target):
        x2 = output.pow(2).sum(axis=1)
        xy = 2*output.mm(output.T)
        dist = (((x2 - xy).T + x2))
        n = output.shape[0]
        loss = (dist - target).pow(2).sum()/(n*(n-1))
        return loss

In [121]:
DistMSELoss()(out.squeeze(2), y)

tensor(109.1038, grad_fn=<DivBackward0>)